In [1]:
#@title 최종 AI 추천 서버 실행 (지능형 폴백 + 상세 정보 입력 통합)

# --- 1. 필수 라이브러리 설치 ---
# T4 GPU 환경에 맞게 bitsandbytes를 포함하여 설치합니다.
!pip install -q -U transformers peft accelerate trl datasets huggingface_hub fastapi "uvicorn[standard]" pyngrok bitsandbytes "pandas==2.2.2"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 503.6/503.6 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 563.3/563.3 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.0/96.0 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.8/510.8 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 81.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.2/452.2 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 4.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not cur

In [2]:
# --- 2. Google Drive 마운트 및 경로 설정 ---
from google.colab import drive
import os
print("📂 Google Drive를 마운트합니다...")
drive.mount('/content/drive')
PROJECT_DIR = "/content/drive/MyDrive/Colab Notebooks/fashion_project"
os.chdir(PROJECT_DIR)
print(f"✅ 작업 디렉토리를 '{PROJECT_DIR}'(으)로 변경했습니다.")


📂 Google Drive를 마운트합니다...
Mounted at /content/drive
✅ 작업 디렉토리를 '/content/drive/MyDrive/Colab Notebooks/fashion_project'(으)로 변경했습니다.


In [3]:
# --- 3. Hugging Face 로그인 ---
from huggingface_hub import login
print("\n--- 🔐 Hugging Face 로그인이 필요합니다 ---")
login()
print("✅ 로그인 성공!")



--- 🔐 Hugging Face 로그인이 필요합니다 ---


✅ 로그인 성공!


In [4]:
# --- 4. 모델 폴더 압축 해제 ---
ZIP_FILE = "gemma-fashion-dpo-final-v3.zip"
ADAPTER_FOLDER = "gemma-fashion-dpo-final-v3"
if not os.path.exists(ADAPTER_FOLDER):
    print(f"\n'{ZIP_FILE}' 파일의 압축을 해제합니다...")
    !unzip -q -o "{ZIP_FILE}" -d .
else:
    print(f"\n'{ADAPTER_FOLDER}' 폴더가 이미 존재합니다.")
ADAPTER_PATH = os.path.join(PROJECT_DIR, ADAPTER_FOLDER)
print(f"✅ 모델 경로 확인: {ADAPTER_PATH}")



'gemma-fashion-dpo-final-v3' 폴더가 이미 존재합니다.
✅ 모델 경로 확인: /content/drive/MyDrive/Colab Notebooks/fashion_project/gemma-fashion-dpo-final-v3


In [5]:
# --- 5. 최종 serve_final.py 파일 자동 생성 ---
%%writefile serve_final.py

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
import pandas as pd
from itertools import product
import time
import random
from fastapi import FastAPI
from pydantic import BaseModel, Field
from typing import List, Dict, Any
import uvicorn
import logging

# --- 유틸리티: 문자열 정규화 ---
def _norm(s):
    if s is None:
        return ""
    return str(s).strip()

def _title(s):
    return _norm(s).title()

# 시즌 표기 통일
_SEASON_MAP = {
    "Autumn": "Fall",
    "All": "All Season",
    "All Season": "All Season",
    "Spring ": "Spring",
    "Summer ": "Summer",
    "Winter ": "Winter",
    "Fall ": "Fall"
}

def _norm_season(s):
    t = _title(s)
    return _SEASON_MAP.get(t, t)

# DataFrame → dict 변환 후 정규화
def _normalize_items(d):
    for v in d.values():
        v["season"] = _norm_season(v.get("season"))
        v["usage"] = _title(v.get("usage"))
        v["subCategory"] = _title(v.get("subCategory"))
        v["masterCategory"] = _title(v.get("masterCategory"))
        v["articleType"] = _title(v.get("articleType"))
    return d

# --- 이벤트 → usage 매핑 ---
_FORMAL = {
    "Office Meeting", "Business Presentation", "Job Interview", "Formal Gala",
    "Wedding Ceremony", "Fine Dining", "Conference Attendance", "Opera Night",
    "Traditional Ceremony", "Public Speaking Event", "Attending A Play"
}
_SPORTS = {
    "Gym Workout", "Hiking", "Charity Marathon", "Cycling Tour", "Game Day At Stadium",
    "Ski Trip", "Beach Volleyball", "Yoga Class"
}

def _usage_for_event(e):
    e = _title(e)
    if e in _FORMAL:
        return ["Formal", "Smart Casual"]
    if e in _SPORTS:
        return ["Sports", "Active"]
    return ["Casual", "Smart Casual"]

# --- 날씨 조건 보정 ---
# --- 날씨명 정규화 ---
def _norm_condition(cond):
    c = (cond or "").strip().lower()
    if c in ("unknown", ""): return "unknown"
    if c in ("clear",): return "clear"
    if c in ("cloudy",): return "cloudy"
    if c in ("fog",): return "fog"
    if c in ("rain", "shower"): return "rain"        # 샤워는 실질적 비로 취급
    if c in ("snow",): return "snow"
    if c in ("thunderstorm",): return "thunderstorm"
    return "other"

# --- 날씨 조건 보정(강화) ---
def _weather_bonus(condition, articleType, part):
    c = _norm_condition(condition)
    at = (articleType or "").lower()
    p  = (part or "").lower()
    bonus = 0.0

    if c == "rain":
        if p == "outerwear" and ("rain" in at or "jacket" in at or "coat" in at or "trench" in at):
            bonus += 0.8
        if p == "footwear" and ("boot" in at):
            bonus += 0.5
        if p == "footwear" and ("sneaker" in at):
            bonus -= 0.4
    elif c == "snow":
        if p == "outerwear": bonus += 0.5
        if p == "footwear" and ("boot" in at): bonus += 0.6
        if p == "topwear" and ("sweater" in at or "hoodie" in at): bonus += 0.2
    elif c == "thunderstorm":
        if p == "outerwear" and ("jacket" in at or "coat" in at): bonus += 0.5
        if p == "footwear" and ("boot" in at): bonus += 0.4
        if p == "footwear" and ("sneaker" in at): bonus -= 0.3
    elif c == "fog":
        if p == "outerwear": bonus += 0.15
    # cloudy/clear/unknown/other → 보정 없음
    return bonus

# --- 온도-시즌 보정(세분화) ---
def _temp_season_bonus(temp, season):
    s = (season or "").title()
    t = float(temp)

    # 온도 구간별 계절 가중치 테이블
    if t <= 0:   # 한겨울
        table = {
            "Winter": +0.70, "Fall": +0.40, "Spring": -0.40, "Summer": -0.80, "All Season": +0.10
        }
    elif t <= 5:  # 겨울에 가까움
        table = {
            "Winter": +0.60, "Fall": +0.30, "Spring": -0.20, "Summer": -0.70, "All Season": +0.10
        }
    elif t <= 10:  # 쌀쌀한 가을/이른 봄
        table = {
            "Winter": +0.40, "Fall": +0.35, "Spring": +0.15, "Summer": -0.50, "All Season": +0.05
        }
    elif t <= 15:  # 전형적인 가을/봄
        table = {
            "Fall": +0.30, "Spring": +0.30, "Winter": +0.10, "Summer": -0.30, "All Season": +0.05
        }
    elif t <= 20:  # 쾌적한 봄/가을
        table = {
            "Spring": +0.40, "Fall": +0.40, "Summer": +0.05, "Winter": -0.30, "All Season": +0.05
        }
    elif t <= 25:  # 여름에 가까움
        table = {
            "Summer": +0.30, "Spring": +0.20, "Fall": +0.10, "Winter": -0.40, "All Season": +0.05
        }
    elif t <= 30:  # 더움
        table = {
            "Summer": +0.50, "Spring": +0.10, "Fall": -0.10, "Winter": -0.50, "All Season": +0.10
        }
    else:  # 한여름
        table = {
            "Summer": +0.70, "Spring": -0.20, "Fall": -0.30, "Winter": -0.70, "All Season": +0.10
        }

    return table.get(s, 0.0)



# --- 기본 설정 ---
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
app = FastAPI()

# --- 전역 변수 ---
model, tokenizer, styles_info, positive_token_id, negative_token_id = None, None, None, None, None
device = "cuda" if torch.cuda.is_available() else "cpu"

# --- 서버 시작 시 모델 로딩 ---
@app.on_event("startup")
def load_resources():
    global model, tokenizer, styles_info, positive_token_id, negative_token_id
    logger.info("✅ 최종 DPO 모델 로딩을 시작합니다...")
    adapter_path = "gemma-fashion-dpo-final-v3"
    csv_path = "styles.csv"

    logger.info(f"Using adapter path: {adapter_path}")
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
    )
    base_model = AutoModelForCausalLM.from_pretrained(
        "google/gemma-2b-it",
        quantization_config=quantization_config,
        device_map="auto"
    )
    model = PeftModel.from_pretrained(base_model, adapter_path)
    model.eval()
    tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
    tokenizer.pad_token = tokenizer.eos_token
    positive_token_id = tokenizer.encode("긍정적", add_special_tokens=False)[0]
    negative_token_id = tokenizer.encode("부정적", add_special_tokens=False)[0]
    logger.info("✅ 모델 및 토크나이저 로딩 완료!")

    try:
        styles_df = pd.read_csv("styles.csv", on_bad_lines='skip', dtype={'id': int}).set_index('id')
        styles_info = styles_df.to_dict('index')
        logger.info("✅ styles.csv 로드 완료!")
    except FileNotFoundError:
        logger.warning("⚠️ styles.csv 파일을 찾을 수 없어, 빈 데이터로 시작합니다.")
        styles_info = {}

# --- 헬퍼 함수 및 데이터 모델 ---
def get_item_description(item_id, temp_styles_info):
    info = temp_styles_info.get(item_id, {})
    season = info.get('season', 'All')
    return f"{info.get('baseColor', '')} {info.get('articleType', '')} ({season})".strip()

class ClosetItem(BaseModel):
    id: int
    gender: str
    masterCategory: str
    subCategory: str
    articleType: str
    baseColor: str
    season: str
    usage: str

class OutfitRequest(BaseModel):
    closet: List[ClosetItem]
    event: str
    temperature: float
    condition: str
    gender: str

@app.post("/recommend_outfit")
def recommend_outfit(req: OutfitRequest):
    start_time = time.time()

    # 1. 입력 데이터 처리
    request_closet_info = {item.id: item.dict() for item in req.closet}
    temp_styles_info = styles_info.copy()
    temp_styles_info.update(request_closet_info)
    temp_styles_info = _normalize_items(temp_styles_info)  # ★ 정규화 추가
    closet_ids = [item.id for item in req.closet]

    # 2. 시즌 필터링(4분할)
    temp = float(req.temperature)
    if temp <= 3:
        suitable_seasons = {"Winter", "All Season"}
    elif temp <= 10:
        suitable_seasons = {"Winter","Fall", "All Season"}
    elif temp <= 15:
        suitable_seasons = {"Fall","Spring", "All Season"}
    elif temp <= 20:
        suitable_seasons = {"Spring", "Summer","Fall", "All Season"}
    elif temp <= 25:
        suitable_seasons = {"Spring", "Summer", "All Season"}
    else:
        suitable_seasons = {"Summer", "All Season"}


    # 3. 이벤트 기반 usage 결정
    target_usage = _usage_for_event(req.event)

    filtered_closet_ids = [
        item_id for item_id in closet_ids
        if item_id in temp_styles_info and
           (temp_styles_info[item_id].get('season') in suitable_seasons) and
           (temp_styles_info[item_id].get('usage') in target_usage)
    ]
    if not filtered_closet_ids:
        return {"error": "현재 상황에 맞는 아이템이 옷장에 없습니다."}

    # 4. 개별 아이템 AI 평가
    item_prompts = [
        f"상황: {req.gender}, {req.temperature}°C, {req.condition}, {req.event}\n"
        f"옷: {get_item_description(item_id, temp_styles_info)}\n결과:"
        for item_id in filtered_closet_ids
    ]
    inputs = tokenizer(item_prompts, return_tensors="pt", padding=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    sequence_lengths = inputs['attention_mask'].sum(dim=1) - 1
    last_token_logits = outputs.logits[torch.arange(len(filtered_closet_ids)), sequence_lengths]
    logits_for_scoring = last_token_logits[:, [negative_token_id, positive_token_id]]
    probs = torch.softmax(logits_for_scoring, dim=-1)
    item_scores = probs[:, 1].tolist()

    # ★ 5.기온 + 날씨 보정(강화)
    OUTER_SET = {'Jackets','Blazers','Waistcoat','Coat','Shrug','Cardigan','Nehru Jackets','Rain Jacket','Sweaters','Bomber Jackets','Overshirts','Trench Coats','Denim Jackets','Light Cardigans'}
    FULL_SET  = {'Dresses','Jumpsuit'}

    for i, item_id in enumerate(filtered_closet_ids):
        info = temp_styles_info[item_id]
        at = info.get("articleType", "")
        sc = info.get("subCategory", "")
        mc = info.get("masterCategory", "")
        season_i = info.get("season")

        # 파트 판정
        if sc == "Topwear":
            part = "Topwear"
        elif sc == "Bottomwear":
            part = "Bottomwear"
        elif mc == "Footwear" or "Shoes" in sc:
            part = "Footwear"
        elif at in OUTER_SET:
            part = "Outerwear"
        elif at in FULL_SET:
            part = "Full Body"
        else:
            part = ""

        t_bonus = _temp_season_bonus(temp, season_i)
        w_bonus = _weather_bonus(req.condition, at, part)
        item_scores[i] = max(0.0, min(1.0, item_scores[i] + t_bonus + w_bonus))

    scored_items = {filtered_closet_ids[i]: item_scores[i] for i in range(len(filtered_closet_ids))}

    # 6. 부위별 분류 및 Top-K 선정
    classified_items = {'Topwear': [], 'Bottomwear': [], 'Outerwear': [], 'Footwear': [], 'Full Body': []}
    outerwear_types = {
        'Jackets','Blazers','Waistcoat','Coat','Shrug','Cardigan','Nehru Jackets',
        'Rain Jacket','Sweaters','Bomber Jackets','Overshirts','Trench Coats',
        'Denim Jackets','Light Cardigans'
    }
    full_body_types = {'Dresses','Jumpsuit'}

    for item_id in filtered_closet_ids:
        info = temp_styles_info[item_id]
        at = info.get('articleType','')
        sc = info.get('subCategory','')
        mc = info.get('masterCategory','')
        part = None
        if at in full_body_types:
            part = 'Full Body'
        elif at in outerwear_types:
            part = 'Outerwear'
        elif sc == 'Topwear':
            part = 'Topwear'
        elif sc == 'Bottomwear':
            part = 'Bottomwear'
        elif mc == 'Footwear' or 'Shoes' in sc:
            part = 'Footwear'
        if part:
            classified_items[part].append(item_id)

    top_items_by_part = {
        part: sorted(items, key=lambda x: scored_items.get(x, 0), reverse=True)[:5]
        for part, items in classified_items.items()
    }

    # --- 조합 생성 ---
    def generate_combinations(items_pool):
        # 신발 포함 조합
        base = list(product(items_pool.get('Topwear', []),
                            items_pool.get('Bottomwear', []),
                            items_pool.get('Footwear', [])))
        outer = list(product(items_pool.get('Topwear', []),
                            items_pool.get('Bottomwear', []),
                            items_pool.get('Outerwear', []),
                            items_pool.get('Footwear', [])))
        full_body = list(product(items_pool.get('Full Body', []),
                                items_pool.get('Footwear', [])))

        combos = base + outer + full_body

        # ★ fallback: 신발이 전혀 없을 경우에만 대체 허용
        if not combos:
            combos = list(product(items_pool.get('Topwear', []),
                                  items_pool.get('Bottomwear', [])))
        return [c for c in combos if c]


    all_combinations = generate_combinations(top_items_by_part)

    if not all_combinations:
        logger.warning("상위 아이템으로 조합 생성 실패. 필터링된 전체 아이템으로 재시도합니다.")
        all_combinations = generate_combinations(classified_items)

    if not all_combinations:
        return {"error": "옷장의 아이템들로 유효한 조합을 만들 수 없습니다."}

    # 7. 최종 조합 평가
    prompts = [
        f"상황: {req.gender}, {req.temperature}°C, {req.condition}, {req.event}\n"
        f"옷: {', '.join([get_item_description(id, temp_styles_info) for id in combo])}\n결과:"
        for combo in all_combinations
    ]
    inputs = tokenizer(prompts, return_tensors="pt", padding=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    sequence_lengths = inputs['attention_mask'].sum(dim=1) - 1
    last_token_logits = outputs.logits[torch.arange(len(all_combinations)), sequence_lengths]
    logits_for_scoring = last_token_logits[:, [negative_token_id, positive_token_id]]
    probs = torch.softmax(logits_for_scoring, dim=-1)
    all_scores = probs[:, 1].tolist()

    best_score_index = all_scores.index(max(all_scores))
    best_score = all_scores[best_score_index]
    best_combo = all_combinations[best_score_index]
    best_combo_ids = [int(id_val) for id_val in best_combo if id_val is not None]
    outfit_str = ", ".join([get_item_description(id, temp_styles_info) for id in best_combo_ids])
    best_outfit_info = {"description": outfit_str, "ids": best_combo_ids}

    explanation = f"긍정적: {req.temperature}°C의 {req.condition} 날씨에 진행되는 {req.event}에 가장 잘 어울리는 스타일입니다."
    end_time = time.time()

    return {
        "best_combination": best_outfit_info,
        "explanation": explanation,
        "best_score": best_score,
        "processing_time": end_time - start_time
    }


Overwriting serve_final.py


In [ ]:
# --- 6. 서버 실행 및 ngrok 터널 생성 ---
import threading, time, requests, uvicorn
from pyngrok import ngrok

def run_app():
  uvicorn.run("serve_final:app", host="0.0.0.0", port=8000, log_level="info")

thread = threading.Thread(target=run_app)
thread.start()
print("\n✅ FastAPI 서버가 백그라운드에서 실행을 시작했습니다...")

print("⏳ 서버가 모델을 로딩하고 준비될 때까지 대기 중입니다...")
start_time = time.time()
server_ready = False
while time.time() - start_time < 600:
    try:
        response = requests.get("http://localhost:8000/", timeout=5)
        if response.status_code == 404:
            print("✅ 서버가 응답하기 시작했습니다!")
            server_ready = True
            break
    except requests.exceptions.ConnectionError: time.sleep(5); print("...")
if not server_ready:
    print("❌ 10분이 지나도 서버가 준비되지 않았습니다. 런타임을 재시작하고 다시 시도해주세요.")
else:
    NGROK_TOKEN = "31HHkLEWGt90qDRoAWd6BqiGL9K_5fJsXN7LgijbtcAVwkwAS" # 🚨
    ngrok.set_auth_token(NGROK_TOKEN)
    ngrok.kill()
    public_url = ngrok.connect(8000)
    print(f"\n🎉 서버가 성공적으로 생성되었습니다. API 주소: {public_url}")
    print("이제 이 주소를 test_api.py에 넣고 로컬에서 테스트를 실행하세요.")

thread.join()


✅ FastAPI 서버가 백그라운드에서 실행을 시작했습니다...
⏳ 서버가 모델을 로딩하고 준비될 때까지 대기 중입니다...
...
...
...
...
...


INFO:     Started server process [609]
INFO:     Waiting for application startup.
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

...

...


tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


...
INFO:     127.0.0.1:55914 - "GET / HTTP/1.1" 404 Not Found
✅ 서버가 응답하기 시작했습니다!

🎉 서버가 성공적으로 생성되었습니다. API 주소: NgrokTunnel: "https://3e1f93043fdf.ngrok-free.app" -> "http://localhost:8000"
이제 이 주소를 test_api.py에 넣고 로컬에서 테스트를 실행하세요.
INFO:     58.72.42.91:0 - "POST /recommend_outfit HTTP/1.1" 200 OK
INFO:     58.72.42.91:0 - "POST /recommend_outfit HTTP/1.1" 200 OK
INFO:     58.72.42.91:0 - "POST /recommend_outfit HTTP/1.1" 200 OK
INFO:     58.72.42.91:0 - "POST /recommend_outfit HTTP/1.1" 200 OK
INFO:     58.72.42.91:0 - "POST /recommend_outfit HTTP/1.1" 200 OK
INFO:     58.72.42.91:0 - "POST /recommend_outfit HTTP/1.1" 200 OK
INFO:     58.72.42.91:0 - "POST /recommend_outfit HTTP/1.1" 200 OK
INFO:     58.72.42.91:0 - "POST /recommend_outfit HTTP/1.1" 200 OK
